In [9]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, ConcatDataset
import torchvision.transforms.functional as F
from torchvision import transforms, datasets
from transformers import AutoModelForImageClassification, AutoProcessor 

model_20000 = AutoModelForImageClassification.from_pretrained("./models/checkpoint_20000")
processor_20000 = AutoProcessor.from_pretrained("./models/checkpoint_20000")


model_4000 = AutoModelForImageClassification.from_pretrained("./models/checkpoint_4000")
processor_4000 = AutoProcessor.from_pretrained("./models/checkpoint_4000")


model_final = AutoModelForImageClassification.from_pretrained("./models/watchdog_ai")
processor_final = AutoProcessor.from_pretrained("./models/watchdog_ai")

In [16]:
IMAGE_SIZE = 224

unnorm = transforms.Normalize(
    mean=[-1.0, -1.0, -1.0],
    std=[2.0, 2.0, 2.0]
)

transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

mncai_dataset = datasets.ImageFolder(root="./data/mncai/train", transform=transform)
CIFAKE_test_dataset = datasets.ImageFolder(root="./data/CIFAKE/test", transform=transform)

combined_dataset = ConcatDataset([mncai_dataset, CIFAKE_test_dataset])
test_loader = DataLoader(combined_dataset, batch_size=32, shuffle=False)

In [17]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [ ]:
from sklearn.metrics import accuracy_score

threshold = 0.85

model_20000.eval()
model_20000.to(device)

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        images, labels = batch  # images: (B, 3, H, W)

        # Unnormalize and convert to PIL
        pil_images = [F.to_pil_image(unnorm(img)) for img in images]

        # Preprocess using the processor
        inputs = processor_20000(images=pil_images, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # Forward pass
        outputs = model_20000(**inputs)
        logits = outputs.logits

        # Get positive class probability
        probs = torch.softmax(logits, dim=-1)
        positive_probs = probs[:, 1]

        # Stricter threshold
        preds = (positive_probs > threshold).int()

        # Store predictions and labels
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy at threshold {threshold}: {accuracy:.4f}")


  0%|          | 0/1250 [00:00<?, ?it/s]/Users/briankwon/Desktop/DSAN 6500/WatchdogAI/.venv/lib/python3.13/site-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'padding'
  return self.preprocess(images, **kwargs)
100%|██████████| 1250/1250 [23:56<00:00,  1.15s/it]

Accuracy at threshold 0.85: 0.7800


In [33]:
threshold = 0.85

model_4000.eval()
model_4000.to(device)

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        images, labels = batch  # images: (B, 3, H, W)

        # Unnormalize and convert to PIL
        pil_images = [F.to_pil_image(unnorm(img)) for img in images]

        # Preprocess using the processor
        inputs = processor_4000(images=pil_images, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        labels = labels.to(device)

        # Forward pass
        outputs = model_4000(**inputs)
        logits = outputs.logits

        # Get positive class probability
        probs = torch.softmax(logits, dim=-1)
        positive_probs = probs[:, 1]

        # Stricter threshold
        preds = (positive_probs > threshold).int()

        # Store predictions and labels
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy at threshold {threshold}: {accuracy:.4f}")

100%|██████████| 1250/1250 [29:57<00:00,  1.44s/it]

Accuracy at threshold 0.85: 0.7759


In [4]:
from PIL import Image
import torch

# Load image
image = Image.open("data/CIFAKE/test/REAL/0000 (3).jpg").convert("RGB")

# Preprocess with processor
inputs = processor_20000(images=image, return_tensors="pt")

# Send model and inputs to device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model_20000.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Get logits (raw predictions)
with torch.no_grad():
    outputs = model_20000(**inputs)
    logits = outputs.logits

# Get predicted class index
predicted_class_idx = logits.argmax(-1).item()

# Optional: map to class label if available
label = model_20000.config.id2label[predicted_class_idx]
print(f"Predicted label: {label}")


Predicted label: REAL
